In [1]:
# gptname = 'gpt2' #small
gptname = 'gpt2-xl'
enablenews = True
colab = True
enabletrash = True
devicename = 'cuda'
lrmult = 0.1
accumn = 10
if not colab:
    filepath = '.'
else:
    filepath = '/content/drive/MyDrive/bio'
    ! pip3 install transformers
    from google.colab import drive
    drive.mount('/content/drive')

import sys
sys.path.append(filepath + '/utils')
sys.path.append(filepath + '/dataset')

     |████████████████████████████████| 2.3MB 14.4MB/s 
     |████████████████████████████████| 901kB 49.1MB/s 
     |████████████████████████████████| 3.3MB 50.1MB/s 
Mounted at /content/drive


In [2]:
import argparse, json, os, pickle, random, time, numpy as np, transformers, torch

from transformers import GPT2Config, GPT2LMHeadModel,AdamW, GPT2Tokenizer
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tnrange, tqdm

from utils import SummarizationDataset, FineTuningDataset, get_tokenizer, generate_sample, sample_seq, set_seed, top_k_top_p_filtering

In [3]:
#please change default arguments if needed

parser = argparse.ArgumentParser()

parser.add_argument("--num_train_epochs",default=1, type=int,  help="no of epochs of training")

parser.add_argument("--seed",default=42, type=int,  help="seed to replicate results")
parser.add_argument("--num_workers",default=2, type=int,  help="num of cpus available")
parser.add_argument("--device",default=torch.device(devicename), help="torch.device object")
parser.add_argument("--model_dir",default= filepath + '/weights', type=str,  help="path to save trained model")
parser.add_argument("--root_dir",default= filepath + '/bignews/gpt2_1024_data', type=str, help="location of json dataset.")
parser.add_argument("--ids_file",default= filepath + '/bignews/ids.json', type=str, help="location of train, valid and test file indexes")
args = parser.parse_args([])

In [4]:
if enablenews:
  strnews = 'ye'
else:
  strnews = 'no'

if enabletrash:
  strtrsh = 'ye'
else:
  strtrsh = 'no'

model_file = os.path.join(args.model_dir,  'model_bio_{}_lr{}_accum{}_{}news_{}trash.bin'.format(
          gptname, int(lrmult*100), accumn, strnews, strtrsh))
config_file = os.path.join(args.model_dir, 'config_bio_{}_lr{}_accum{}_{}news_{}trash.json'.format(
          gptname, int(lrmult*100), accumn, strnews, strtrsh))

config = GPT2Config.from_json_file(config_file)
tokenizer = get_tokenizer(gptname)
model = GPT2LMHeadModel(config)
state_dict = torch.load(model_file, map_location=args.device)
_ = model.load_state_dict(state_dict)
_ = model.eval()
_ = model.to(args.device)

In [5]:
valid_data = FineTuningDataset(filepath + '/dataset', tokenizer, mode='valid', addnews=enablenews, length=10)
print(valid_data.mode)
print(valid_data.len)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
valid
10


In [6]:
for i in range(10):
  idx = valid_data[i]['sum_idx']
  article = valid_data[i]['article'][:idx+1].tolist()
  summary = valid_data[i]['article'][idx+1:][:100].tolist()

  print('new_article', end='\n\n')
  print(tokenizer.decode(article + tokenizer.encode(valid_data[i]['name'])), end='\n\n')
  print('actual_summary', end='\n\n')
  print(tokenizer.decode(summary, skip_special_tokens=True), end='\n\n')
  print("Generated Summary: \n")
  generated_text = sample_seq(
      model, article + tokenizer.encode(valid_data[i]['name']), 100, args.device, 
      #temperature=2, top_k=5, top_p=0.5
      temperature=3, top_k=5, top_p=0.5)[0]
  print(tokenizer.decode(
      generated_text[len(article):], skip_special_tokens=True))
  print("------------------------------------------------------------ \n")
    

new_article

Alexander Bortnikov <|sep|> The EU has added 15 individuals and 18 entities to its sanctions list targeting Russians linked to the separatist uprising in eastern Ukraine. Igor Sechin (US sanctions) The head of Russia's leading integrated oil and gas company Rosneft has shown utter loyalty to Mr Putin, according to US officials. Sergei Chemezov (EU and US sanctions) The director of the Rostec high-technology state corporation, Mr Chemezov has known Mr Putin since the 1980s when they lived in the same apartment complex in East Germany. He is also the personal banker for senior officials of the Russian Federation, including Mr Putin, according to the US Treasury. Ramzan Kadyrov (EU sanctions) The Russian-backed President of Chechnya has voiced strong support for the rebels and for Russia's annexation of Crimea. Sergei Glazyev (US and EU sanctions) Widely regarded as Mr Putin's chief adviser on Ukraine, he talked openly of intervening militarily a month before Russian troops m